In [1]:
import numpy as np
import pandas as pd
import souper_v2
from tqdm import tqdm_notebook

In [2]:
sampled_players = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_list_of_sampled_players.csv')

In [3]:
sampled_players.head()

,player,lastname,team,link_tm,link_ws
0,Max Clark,Clark,Vitesse,/max-clark/profil/spieler/183291,/Players/313996/Show/Max-Clark
1,Sergio Postigo,Postigo,Levante,/sergio-postigo/profil/spieler/158791,/Players/109896/Show/Sergio-Postigo
2,Stanislav Iljutcenko,Iljutcenko,Duisburg,/stanislav-iljutcenko/profil/spieler/191292,/Players/132901/Show/Stanislav-Iljutcenko
3,Iván Marcone,Marcone,Boca Juniors,/ivan-marcone/profil/spieler/90451,/Players/125246/Show/Iván-Marcone
4,Florent Mollet,Mollet,Montpellier,/florent-mollet/profil/spieler/222859,/Players/114169/Show/Florent-Mollet


In [4]:
sp = souper_v2.souper()

In [8]:
def obtain_national_ap_and_agent(players_df):
    
    links = players_df.link_tm
    nationality = ""
    
    returner = {'link': [], 'nationality': [], 'minutes': [], 'agent': []}
    
    for each_link in tqdm_notebook(links):
        
        # Obtaining final link
        back_link = each_link.replace("profil","nationalmannschaft")
        new_link = 'https://www.transfermarkt.co.uk' + back_link
        print(new_link)
        soup = sp.get_soup(new_link)
        
        # Obtaining nationality
        try:
            nationality = soup.find('span', attrs={'itemprop':'nationality'}).text.strip()
        except:
            nationality = np.nan
        
        # Obtaining minutes
        national_team = [x.text.strip() for x in soup.find_all('a', attrs={'class': 'chzn-single'}) if x.text.strip() == nationality]
        if len(national_team) == 0:
            national_team = ""
        else:
            national_team = national_team[0]
        
        # Checking if list corresponds to nationality. Otherwise is a minor national team
        if national_team == nationality:
            minutes_list = []
            for each in soup.find_all('div', attrs={'id':'yw1'}):
                for minutes in each.find_all('td', attrs={'class': 'rechts'}):
                    minutes_list.append(minutes.text.strip())

            final_list = [int(x.replace(".","").replace("'","")) for x in minutes_list if x.replace("'","").replace(".","").isnumeric()]
            final_list.sort(reverse=True)
            if len(final_list) == 0:
                minutes = 0
            else:
                minutes = final_list[0]
            
        else:
            minutes = 0
            
        # Obtaining agent   
        values = [x.text.strip() for x in soup.find_all('span', attrs={'class':'dataValue'})]
        items = [x.text.strip() for x in soup.find_all('span', attrs={'class':'dataItem'})]
        agent = [x for x,y in zip(values,items) if y == "Agent:"]
        if len(agent) == 0:
            agent = "No agent"
        else:
            agent = agent[0]
        
        returner['link'].append(each_link)
        returner['nationality'].append(nationality)
        returner['minutes'].append(minutes)
        returner['agent'].append(agent)
    
        
    return returner

In [ ]:
national_apps = obtain_national_ap_and_agent(sampled_players)

In [10]:
national_apps_df = pd.DataFrame(national_apps)

In [12]:
national_apps_df.to_csv(r'/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_national_apps.csv',index=False)

In [13]:
test_loading = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_national_apps.csv')

In [14]:
test_loading.head()

,link,nationality,minutes,agent
0,/max-clark/profil/spieler/183291,England,0,KIN
1,/sergio-postigo/profil/spieler/158791,Spain,0,JPA Soccer
2,/stanislav-iljutcenko/profil/spieler/191292,Russia,0,Coaches & More GmbH
3,/ivan-marcone/profil/spieler/90451,Argentina,0,Christian Bragarnik
4,/florent-mollet/profil/spieler/222859,France,0,No agent
